# Scrape Data From Wikipedia

In [ ]:
import pandas as pd
!pip install lxml

In [126]:
#from IPython.display import IFrame
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#IFrame(url, width=800, height=350)

In [150]:
data, = pd.read_html(url, match="Postcode",header = 0)
data.rename(columns = {'Postcode':'PostalCode'}, inplace = True)
data.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [151]:
data = data[data["Borough"] != "Not assigned"]
data.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [153]:
borough_func = lambda s: s.iloc[0]
neighborhood_func = lambda s: ", ".join(s)
agg_funcs = {"Borough": borough_func, "Neighbourhood": neighborhood_func}
data_temp = data.groupby(by="PostalCode").aggregate(agg_funcs)
data_temp.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [154]:
data = data_temp.reset_index()[data.columns]
data.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [155]:
data[data["Neighbourhood"] == "Not assigned"]

,PostalCode,Borough,Neighbourhood
93,M9A,Queen's Park,Not assigned


In [156]:
for (j, row) in data.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        borough = row["Borough"]
        print("Replace and test \"Not assigned\" => %s in row %i" % (borough, j))
        row["Neighbourhood"] = borough

Replace and test "Not assigned" => Queen's Park in row 93


In [157]:
data.shape
data.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Load Geospatial Data

In [105]:
!pip install geopy 
!pip install folium
!pip install geocoder 

In [106]:
locations = pd.read_csv("https://cocl.us/Geospatial_data")
locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [158]:
locations.columns = ["PostalCode", "Latitude", "Longitude"]
data2 = pd.merge(data, locations, on="PostalCode")
data2.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
